In [ ]:
import re
import json
import os
import shutil

from datetime import datetime
import IPython.display as ipd

import numpy as np
import pandas as pd
pd.options.display.max_columns = 100

import datasets as hfd

from transformers import (
    Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor,
    Wav2Vec2ForCTC,
    TrainingArguments, Trainer
)

import torch
print(torch.cuda.is_available())

In [ ]:
from src.data_collator import DataCollatorCTCWithPadding

In [ ]:
import logging
logging_format_str = '%(asctime)s:%(name)s:%(levelname)s:%(message)s'
logging.basicConfig(format=logging_format_str, level=logging.WARNING)
logger = logging.getLogger('STT')
logger.setLevel(level=logging.DEBUG)

In [ ]:
DATA_ROOT_DP = os.environ['DATA_HOME']
CV_PROCESSED_2_DP = f'{DATA_ROOT_DP}/datasets/cv-corpus-8.0-2022-01-19__be__processed__2'

In [ ]:
ds = hfd.load_from_disk(CV_PROCESSED_2_DP)
ds

In [ ]:
wer_metric = hfd.load_metric("wer")

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
processor = Wav2Vec2Processor.from_pretrained('artifacts/processor')

In [ ]:
vocab = processor.tokenizer.get_vocab()
pd.Series(vocab).to_frame().T

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
vocab_size = len(processor.tokenizer.get_vocab())
logger.info(f'vocab_size: {vocab_size}')

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base", 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=vocab_size
)

In [ ]:
model.freeze_feature_extractor()

In [ ]:
now_str = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')

train_out_dp = os.path.join('train', now_str)
os.makedirs(train_out_dp, exist_ok=True)
print(f'will save train output to: "{train_out_dp}"')

logs_dp = os.path.join(train_out_dp, 'logs')
print(f'will save train logs to: "{logs_dp}"')

In [ ]:
training_args = TrainingArguments(
    output_dir=train_out_dp,
    logging_dir=logs_dp,

    num_train_epochs=1,
    evaluation_strategy='steps',
    save_strategy='steps',
    logging_strategy='steps',
    eval_steps=1000,
    logging_steps=1000,
    save_steps=1000,
    save_total_limit=None,
    load_best_model_at_end=True,
    metric_for_best_model='wer',
    greater_is_better=False,

    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    fp16=True,
    group_by_length=False,
    gradient_checkpointing=True, 
    learning_rate=1e-4,
    weight_decay=0.005,
    warmup_steps=700,  # ????
)

In [ ]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=ds['train'],
    eval_dataset=ds['dev'],
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train()